# Easy Agent Tutorial
This notebook file provide three examples of using LLM based agents with different tool sets.

prequisites:
- Python 3.10+
- Install required packages:
  ```bash
  pip install "mcp[cli]" smolagents
  ```

## Task Description

如README所述，该项目应用三种方案，从不同的角度实现了agentic RAG的功能。为了演示，这一次我们将会构建一个信安四大会的查询，来进行感兴趣论文的搜索以及基于题目选择合适的会议进行投稿。

## Data Preparation

In [1]:
# 我们将使用dblp数据集来进行演示。首先下载四大会最近几年的会议论文数据：
import requests
import json
import os

if not os.path.exists("dblp_sec_papers.json"):
    with requests.Session() as sess:
        url = "https://dblp.org/search/publ/api"
        params = {"q": "security", "format": "json"}
        tocs = {
            "ndss": "toc:db/conf/ndss/ndss2025.bht:",
            "sp": "toc:db/conf/sp/sp2025.bht:",
            "ccs": "toc:db/conf/ccs/ccs2025.bht:",
            "usenix": "toc:db/conf/uss/uss2025.bht:",
        }
        papers = []
        for k, v in tocs.items():
            response = sess.get(url, params={"q": v, "h": 1000, "format": "json"})
            data = response.json()
            data = data["result"]["hits"]["hit"]
            papers.extend(data)
        with open(f"dblp_sec_papers.json", "w", encoding="utf-8") as f:
            json.dump(papers, f, ensure_ascii=False, indent=4)
else:
    with open(f"dblp_sec_papers.json", "r", encoding="utf-8") as f:
        papers = json.load(f)
papers = [x["info"] for x in papers]
titles = [f"[{x['venue']} {x['year']}] {x['title']}" for x in papers]

In [5]:
# 创建并保存向量数据库
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
if "db" not in globals():
    db = FAISS.from_texts(titles, OpenAIEmbeddings())
db.save_local("faiss_db")

## 方案0： 一切奇迹的始发点——传统工具调用

在开始之前，先看一下传统的工具调用大概长啥样，有怎样的优缺点

In [7]:
import json
import logging
import re

import dotenv
import tenacity
import yaml
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from openai.types import *
from openai.types.chat import *


def raw_toolcall():
    client = OpenAI()
    db = FAISS.load_local(
        "faiss_db",
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True,
    )
    tools_def = [
        {
            "type": "function",
            "name": "query_db",
            "description": "accept keyword and return related information",
            "parameters": {
                "type": "object",
                "properties": {
                    "kw": {
                        "type": "string",
                        "description": "The keyword to query the database",
                    }
                },
                "required": ["kw"],
            },
        }
    ]

    def real_ask(question: str):
        from openai.types.responses.response_input_param import Message

        input_msgs: list[Message] = [
            {
                "type": "message",
                "role": "system",
                "content": f"You are a helpful research assistant. When given a question, you must first decide if you need to query the database to get relevant information. If so, use the tool 'query_db' with appropriate keywords extracted from the question. After getting the information, provide a comprehensive answer based on both the retrieved information and your own knowledge.",
            },
            {
                "type": "message",
                "role": "user",
                "content": question,
            },
        ]
        resp = client.responses.create(
            model="gpt-5.1",
            tools=tools_def,
            input=input_msgs,
            tool_choice="required",
        )
        print(resp.output)
        for toolcall in resp.output:
            if toolcall.type != "function_call":
                continue
            if toolcall.name == "query_db":
                kw = json.loads(toolcall.arguments)["kw"]
                print(f"Keyword: {kw}")
                docs = []
                for skw in kw.split():
                    if not (skw := skw.strip()):
                        continue
                    print(f"Searching for keyword: {skw}")
                    docs.extend(db.similarity_search(skw, k=30))
                rag_result = "\n".join([doc.page_content for doc in docs])
                input_msgs.append(toolcall)
                input_msgs.append(
                    {
                        "type": "function_call_output",
                        "call_id": toolcall.call_id,
                        "output": str(rag_result),
                    }
                )
                print(f"{kw=} appended.")
        if input_msgs[-1]["type"] == "function_call_output":
            resp = client.responses.create(
                model="gpt-5.1",
                input=input_msgs,
                # tools=tools_def,
                stream=True,
            )
            for chunk in resp:
                if chunk.type == "response.output_text.delta":
                    print(chunk.delta, end="", flush=True)
            print()
        else:
            print(resp.output_text)

    while True:
        inp = input("=> ")
        if not inp.strip():
            break
        real_ask(inp)


raw_toolcall()

[ResponseFunctionToolCall(arguments='{"kw":"blockchain LLM papers"}', call_id='call_ZNk69x2z6uSxuPEyhldVmhRc', name='query_db', type='function_call', id='fc_0edcc91604d6e0a4006938e28cf5c0819282e662d761ad49ea', status='completed')]
Keyword: blockchain LLM papers
Searching for keyword: blockchain
Searching for keyword: LLM
Searching for keyword: papers
kw='blockchain LLM papers' appended.
你这个方向目前还比较前沿，严格意义上“区块链 × LLM”的论文不算多，但已经有几类比较典型的结合方式，可以按“区块链为LLM赋能”和“LLM为区块链赋能”来找文献。我先列代表性论文和关键词，方便你自己去搜（Google Scholar / arXiv / dblp），再给你一个按方向分类的阅读建议。

下面所有英文标题你直接复制去搜索就能找到 PDF。

---

## 一、LLM 为区块链赋能（用 LLM 做智能合约/链上安全/分析）

### 1. 智能合约分析与形式化验证

- **PropertyGPT: LLM-driven Formal Verification of Smart Contracts through Retrieval-Augmented Property Generation**  
  NDSS 2025（网络与分布式系统安全研讨会）  
  关键词：  
  - “LLM-driven Formal Verification of Smart Contracts”  
  - “Retrieval-Augmented Property Generation”  
  核心思路：用 LLM 自动生成合约的安全属性 / 规范，再结合形式化验证工具检查。适合关注“LLM + 智能合约安全”的同学。

- 可一并检索的关键词：  
  - “LLM for smart co

### 传统工具调用的优缺点

- 优点：直接用模型原生的 function/tool 调用协议，链路短、开销低，JSON Schema 参数校验清晰。
- 优点：可以精确控制何时调用工具、使用 `tool_choice` 等参数强制执行，消息格式透明、便于调试和流式输出。
- 优点：依赖少，不绑框架，易于插入到现有服务或与其他编排层组合。
- 缺点：需要手写对话状态管理、工具输入输出拼接，容易出错且样板代码多。
- 缺点：缺少自动规划/多步推理、重试、fallback 等封装能力，复杂流程要自行实现。
- 缺点：与特定模型/协议耦合，换提供方或多模型时需适配；安全性与数据清洗（如反序列化、去重）也要自管。

## SmolAgent::CodeAgent

In [1]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from smolagents import CodeAgent, OpenAIServerModel, tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)

sys_prompt = """
You are a helpful research assistant.
When given a question, you must query the database to get relevant information.
Use the tools with appropriate arguments derived from the question.
After getting the information, provide a comprehensive answer based on both the retrieved information.
Output the final answer in markdown wrapped in final_answer().
""".strip()


@tool
def query_paperdb(kw: str) -> str:
    """
    Accept keywords and return related paper titles, multiple keywords should be separated by '|'.
    This tool should be called before output any realworld-related information.

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split("|"):
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=10))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


agent = CodeAgent(
    model=OpenAIServerModel("gpt-5.1"),
    tools=[query_paperdb],
    stream_outputs=True,
    # use_structured_outputs_internally=True, # True for structured_code_agent.yaml, False for code_agent.yaml
)

# from Gradio_UI import GradioUI

# GradioUI(agent).launch()

res = agent.run(f"{sys_prompt}\n\n{input("=> ")}", return_full_result=True)
print(f"{res.token_usage=} {res.timing=}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a helpful research assistant.                                                                           │
│ When given a question, you must query the database to get relevant information.                                 │
│ Use the tools with appropriate arguments derived from the question.                                             │
│ After getting the information, provide a comprehensive answer based on both the retrieved information.          │
│ Output the final answer in markdown wrapped in final_answer().                                                  │
│                                                                                                                 │
│ 查询llm安全相关的论文                                                                                           │
│                                                                                                                 │
╰─ OpenAIModel - gpt-5.1 ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from typing import List                                                                                          
                                                                                                                   
  # Step 1: Query the paper database for LLM safety related keywords (in English and Chinese)                      
  kw = "LLM safety|large language model safety|AI alignment|红队 攻击 LLM|大模型 安全 防护|jailbreak defense"      
  papers_info = query_paperdb(kw=kw)                                                                               
                                                                                                                   
  print(papers_info)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'from typing import List' due to: InterpreterError: Import from typing is not 
allowed. Authorized imports are: ['queue', 'collections', 'unicodedata', 'datetime', 'stat', 'random', 're', 
'math', 'statistics', 'time', 'itertools'\]

...

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # 这里我们已经拿到了大量与“大语言模型（LLM）安全”高度相关的论文题目                                              
  # 接下来不再调用工具，而是基于这些结果，按主题给你做一个中文综述与阅读建议。                                     
                                                                                                                   
  summary = r"""                                                                                                   
  下面根据你查询到的 llm 安全相关论文列表，按研究方向给出一个结构化综述和阅读建议。                                
  （会议缩写：SP=IEEE S&P，CCS=ACM CCS，NDSS=NDSS，USENIX=USENIX Security）                                        
                                                                                                                   
  ---                                                                                                              
                                                                                                                   
  ## 一、总体图景：LLM 安全/对齐的主要研究方向                                                                     
                                                                                                                   
  从检索结果看，LLM 安全论文大致可以分为几大类：                                                                   
                                                                                                                   
  1. **越狱（Jailbreak）与红队攻击**                                                                               
     - 设计更强的越狱攻击方法，系统性评估主流模型的防御能力。                                                      
     - 提出自动化测试/模糊测试（fuzzing）等技术来发现安全策略的弱点。                                              
     - 同时研究防御策略，例如对齐增强、概念级控制等。                                                              
                                                                                                                   
  2. **提示注入（Prompt Injection）、Prompt 窃取和推断**                                                           
     - 针对多轮对话、RAG、代理（agent）、提示服务等场景下的注入攻击。                                              
     - 提出结构化查询、对齐优化等防御机制。                                                                        
     - 研究 Prompt Stealing / Prompt Inference 等推断攻击。                                                        
                                                                                                                   
  3. **对齐与“安全失配”（Safety Misalignment）**                                                                   
     - 模型尽管被对齐，但在复杂场景中仍会出现不安全输出。                                                          
     - 分析对齐在微调、工具调用、RAG 等场景中的退化问题。                                                          
     - 探讨如何在保持能力的同时，减少安全对齐的“遗失”。                                                            
                                                                                                                   
  4. **隐私与数据泄露**                                                                                            
     - 从模型中提取个人敏感信息（PII）。                                                                           
     - 成员推断（membership inference）与训练数据保护。                                                            
     - 在医疗等敏感领域评估 LLM 的隐私风险与防御。                                                                 
                                                                                                                   
  5. **后门、模型合并与生成数据风险**                                                                              
     - 大模型后门攻击与检测方法。                                                                                  
     - 模型合并（model merging）中的隐含安全风险。                                                                 
     - 使用生成数据微调可能引入伪隐私、偏见和安全问题。                                                            
                                                                                                   

Final answer: 
下面根据你查询到的 llm 安全相关论文列表，按研究方向给出一个结构化综述和阅读建议。  
（会议缩写：SP=IEEE S&P，CCS=ACM CCS，NDSS=NDSS，USENIX=USENIX Security）

---

## 一、总体图景：LLM 安全/对齐的主要研究方向

从检索结果看，LLM 安全论文大致可以分为几大类：

1. **越狱（Jailbreak）与红队攻击**
   - 设计更强的越狱攻击方法，系统性评估主流模型的防御能力。
   - 提出自动化测试/模糊测试（fuzzing）等技术来发现安全策略的弱点。
   - 同时研究防御策略，例如对齐增强、概念级控制等。

2. **提示注入（Prompt Injection）、Prompt 窃取和推断**
   - 针对多轮对话、RAG、代理（agent）、提示服务等场景下的注入攻击。
   - 提出结构化查询、对齐优化等防御机制。
   - 研究 Prompt Stealing / Prompt Inference 等推断攻击。

3. **对齐与“安全失配”（Safety Misalignment）**
   - 模型尽管被对齐，但在复杂场景中仍会出现不安全输出。
   - 分析对齐在微调、工具调用、RAG 等场景中的退化问题。
   - 探讨如何在保持能力的同时，减少安全对齐的“遗失”。

4. **隐私与数据泄露**
   - 从模型中提取个人敏感信息（PII）。
   - 成员推断（membership inference）与训练数据保护。
   - 在医疗等敏感领域评估 LLM 的隐私风险与防御。

5. **后门、模型合并与生成数据风险**
   - 大模型后门攻击与检测方法。
   - 模型合并（model merging）中的隐含安全风险。
   - 使用生成数据微调可能引入伪隐私、偏见和安全问题。

6. **面向特定群体/应用的安全**
   - 面向青少年/未成年人内容安全。
   - 面向代码生成、漏洞检测等安全相关任务的 LLM 应用。
   - 面向搜索、Web 访问、应用商店等“LLM 上线”场景的新风险。

7. **多模态安全（Vision-Language / Text-to-Image）**
   - 图文大模型识别不安全内容的能力评估与改进。
   - 文本到图像模型的越狱与防御。

下面以具体论文为例，给你一个“按方向+代表论文”的阅读清单和简要要点，方便你搭建知识体系。

---

## 二、越狱攻击与防御（Jailbreak / Red-teaming）

**攻击方法类：**

- **[USENIX 2025] TwinBreak: Jailbreaking LLM Security Alignments based on Twin Prompts**  
  - 思路：利用“成对的双胞胎提示（Twin Prompts）”构造更隐蔽、稳定的越狱攻击。
  - 贡献：系统展现现有对齐策略如何在 Twin Prompts 下被绕过，说明安全策略对某些结构化提示不够鲁棒。
  - 你可以关注：提示模式（pattern），对齐策略在结构对称下的失效机理。

- **[USENIX 2025] PAPILLON: Efficient and Stealthy Fuzz Testing-Powered Jailbreaks for LLMs**  
  - 思路：将“模糊测试（fuzzing）”思想引入越狱提示搜索，自动生成多样化攻击提示。
  - 特点：高效率、可自动运行，适合作为安全评估工具链的一部分。

- **[USENIX 2025] Great, Now Write an Article About That: The Crescendo Multi-Turn LLM Jailbreak Attack**  
  - 思路：多轮对话式越狱，先从温和问题逐渐诱导模型突破安全边界。
  - 价值：提示多轮交互引入的新攻击面，单轮审查防御不足。

- **[SP 2025] Fuzz-Testing Meets LLM-Based Agents: An Automated and Efficient Framework for Jailbreaking 
Text-to-Image Generation Models**  
  - 虽是图像生成模型，但方法论（代理+fuzz）可迁移到 LLM 红队自动化。

**防御方法类：**

- **[USENIX 2025] SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical Manner**  
  - 核心：利用 LLM 自身能力构造自我防御管线（如自审查、自修正）。
  - 实践性：强调可部署性，对工程系统启发大。

- **[USENIX 2025] JBShield: Defending Large Language Models from Jailbreak Attacks through Activated Concept 
Analysis and Manipulation**  
  - 关键点：通过“激活概念（activated concepts）”视角理解和控制模型内部对有害内容的表征。
  - 特点：更偏向可解释+内部干预，而非仅靠外部规则过滤。

- **[USENIX 2025] Exploiting Task-Level Vulnerabilities: An Automatic Jailbreak Attack and Defense Benchmarking for
LLMs**  
  - 贡献：不只给攻击，还给出一套评价不同防御配置的基准框架，适合做横向对比实验。

---

## 三、提示注入、提示窃取与结构化防御

**攻击与测量：**

- **[SP 2025] On the Effectiveness of Prompt Stealing Attacks on In-the-Wild Prompts**  
- **[USENIX 2025] PRSA: Prompt Stealing Attacks against Real-World Prompt Services**  
  - 聚焦：通过反向推理、黑盒查询等方式从在线服务中窃取高价值系统提示（system prompt / jailbreak prompt）。
  - 有助于理解 Prompt 作为“知识资产”的威胁模型。

- **[SP 2025] Prompt Inversion Attack Against Collaborative Inference of Large Language Models**  
- **[CCS 2025] Prompt Inference Attack on Distributed Large Language Model Inference Frameworks**  
  - 场景：多方推理、分布式推理中的 prompt 泄露风险。

- **[SP 2025] Fun-tuning: Characterizing the Vulnerability of Proprietary LLMs to Optimization-Based Prompt 
Injection Attacks via the Fine-Tuning Interface**  
  - 看点：利用“微调接口”实现类似 prompt injection 的效果，说明开放微调 API 本身就是一个新的攻击面。

**防御与对齐优化：**

- **[USENIX 2025] StruQ: Defending Against Prompt Injection with Structured Queries**  
  - 思路：将自然语言指令映射为结构化查询/调用，减少模型对“任意文本”的被动执行，限制攻击面。

- **[CCS 2025] SecAlign: Defending Against Prompt Injection with Preference Optimization**  
  - 方法：通过偏好优化（preference optimization）训练模型拒绝 prompt injection 类的恶意模式。

- **[SP 2025] DataSentinel: A Game-Theoretic Detection of Prompt Injection Attacks**  
  - 视角：博弈论建模攻击者/防御者，从策略优化角度设计检测机制。

---

## 四、安全对齐与“(In)Security / Misalignment”问题

- **[NDSS 2025] Safety Misalignment Against Large Language Models**  
  - 重点：系统性分析模型在不同任务/场景下的安全表现与对齐目标的偏差。
  - 有利于理解“为什么看起来合规的模型在某些边缘情况仍危险”。

- **[SP 2025] Alleviating the Fear of Losing Alignment in LLM Fine-tuning**  
  - 关注：下游微调是否、以及如何破

[Step 3: Duration 53.20 seconds| Input tokens: 10,080 | Output tokens: 4,046]

res.token_usage=TokenUsage(input_tokens=10080, output_tokens=4046, total_tokens=14126) res.timing=Timing(start_time=1765350604.0020978, end_time=1765350669.0835783, duration=65.08148050308228)


### 方案说明：SmolAgent::CodeAgent（两种提示模板差异）
- 核心循环：两条线路都遵循多步推理，但输出格式不同。
  - 标准版（`code_agent.yaml`，`use_structured_outputs_internally=False`）：每步输出 Thought→Code→Observation，代码必须包裹在自定义 `{{code_block_opening_tag}}`/`{{code_block_closing_tag}}` 中；中间结果用 `print` 进入 Observation，最终用 `final_answer` 收束。
  - 结构化版（`structured_code_agent.yaml`，`use_structured_outputs_internally=True`）：每步输出固定 JSON `{ "thought": "...", "code": "..." }`，默认不需要自定义 code_block 标签；解析稳定、便于日志/回放。
- 规划 scaffold：两者都要求先做 facts survey + high-level plan（initial_plan / update_plan），但标准版在文本里更强调「分步打印、避免链式依赖」。
- 工具调用与链式策略：
  - 标准版明确区分“有 JSON schema 的工具可以链式调用，非结构化工具应先 `print` 再下一步用”，并警示不要重做相同参数的调用、不要用工具名做变量名。
  - 结构化版主要提供最小约束（定义变量再用、参数名显式），输出以 JSON 承载 Thought 和 Code，便于上层程序直接消费。
- 执行与可读性：
  - 标准版输出包含 Thought/Code/Observation 叙事，适合人工旁观调试、流式展示和教学场景。
  - 结构化版输出紧凑、机器友好，适合日志解析、监控、对接编排器或二次路由。
- 选择建议：
  - 需要确定性结构、便于程序解析/回放/审计时，用 `use_structured_outputs_internally=True`（结构化版）。
  - 需要人类可读的逐步对话、希望看到 code_block 包裹和 Observation 明细，或想依赖模板中的链式提示时，用 `use_structured_outputs_internally=False`（标准版）。
- 常见踩坑提示：
  - 标准版务必保持自定义 code_block 标签，否则解析失败；非结构化工具调用不要在同一块紧接依赖下一工具输出。
  - 结构化版的 JSON 输出中请将可执行代码写在 `code` 字段，仍需显式 `final_answer(...)` 收口；确保不要把工具名当变量名。
- 性能与上下文：标准版提示体积更大，可能略增 token 开销；结构化版更紧凑，节省上下文。